In [ ]:
import cellxgene_census

In [20]:
#census_datasets.loc[tabula_liver,].head(1)

,soma_joinid,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
6,444,Publication: https://doi.org/10.1038/s41586-02...,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,The Tabula Muris Consortium et al. (2020) Nature,4546e757-34d0-4d17-be06-538318925fcd,ab2c1368-ab62-44a2-8bb2-18e8999acd64,Liver - A single-cell transcriptomic atlas cha...,4546e757-34d0-4d17-be06-538318925fcd.h5ad,2859


In [7]:
# Default fallback to prevent huge queries
tissue = "decidua basalis"
cell_type_ontology_term_id = "CL:0000525"
dataset_id = "f171db61-e57e-4535-a06a-35d8b6ef8f2b"
is_primary_data = True
# Mapping of field names to values
filter_args = {
    "dataset_id": dataset_id,
    "cell_type_ontology_term_id": cell_type_ontology_term_id,
    "tissue": tissue
}

In [8]:
# Convert all values to list
filter_args = {k: v if isinstance(v, list) else [v] for k, v in filter_args.items() if v is not None}

# Build value_filter string
filters = []
if is_primary_data:
    filters.append("is_primary_data == True")
for k, v in filter_args.items():
    if v:
        quoted = [f'"{x}"' for x in v]
        filters.append(f"{k} in [{', '.join(quoted)}]")
value_filter = " and ".join(filters) if filters else None
value_filter

'is_primary_data == True and dataset_id in ["f171db61-e57e-4535-a06a-35d8b6ef8f2b"] and cell_type_ontology_term_id in ["CL:0000525"] and tissue in ["decidua basalis"]'

In [ ]:
census_datasets = (census["census_info"]["datasets"].read().concat().to_pandas())

In [34]:
census_datasets.to_csv("census_info_datasets.tsv", sep='\t', index=None)

In [17]:
census = cellxgene_census.open_soma(census_version="latest")
census_datasets = (
    census["census_info"]["datasets"].read(value_filter="collection_name == 'Tabula Muris Senis'").concat().to_pandas()
)
tabula_liver = census_datasets["dataset_title"].str.contains("liver", case=False)
census_datasets.loc[tabula_liver,].head(1)


,soma_joinid,citation,collection_id,collection_name,collection_doi,collection_doi_label,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
6,444,Publication: https://doi.org/10.1038/s41586-02...,0b9d8a04-bb9d-44da-aa27-705bb65b54eb,Tabula Muris Senis,10.1038/s41586-020-2496-1,The Tabula Muris Consortium et al. (2020) Nature,4546e757-34d0-4d17-be06-538318925fcd,ab2c1368-ab62-44a2-8bb2-18e8999acd64,Liver - A single-cell transcriptomic atlas cha...,4546e757-34d0-4d17-be06-538318925fcd.h5ad,2859


In [ ]:
species="homo_sapiens"
obs_df = cellxgene_census.get_obs(
            census,
            species,
            column_names=["dataset_id", "tissue_general"]
        )

obs_df = obs_df.drop_duplicates()
#.to_csv("census_info_id_per_tissue.tsv", sep='\t', index=None)



In [ ]:
# 3. Aggregate tissues by dataset_id
tissue_map = (
    obs_df.dropna(subset=["tissue_general"])
    .groupby("dataset_id", observed=False)["tissue_general"]
    .unique()
    .reset_index()
)
tissue_map["tissue_general"] = tissue_map["tissue_general"].apply(list)

# 4. Merge tissue info into datasets
merged = pd.merge(census_datasets, tissue_map, on="dataset_id", how="left")


/tmp/ipykernel_47729/2498269174.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("dataset_id")["tissue_general"]


NameError: name 'pd' is not defined

In [ ]:
    with cellxgene_census.open_soma(census_version=census_version) as census:

        obs_df = cellxgene_census.get_obs(
            census,
            species,
            value_filter=value_filter,
            column_names=column_names
        ).head(100)  # Limit to 100 rows to avoid timeout

    dataset_ids = sorted(obs_df["dataset_id"].dropna().unique().tolist())

In [6]:
import os
import httpx
from typing import Optional

In [14]:
async def make_hubmap_endpoint_request_tbd(endpoint: str, query_param: dict) -> Optional[dict]:
    """
    Query HuBMAP API v3 and return the JSON response.

    Args:
        endpoint (str): HuBMAP API endpoint path (e.g., "search", "entities/{id}", etc.).
        query_param (dict): Dictionary of query parameters.

    Returns:
        Optional[dict]: Parsed JSON response from HuBMAP API, or None if an error occurred.
    """
    try:
        async with httpx.AsyncClient(timeout=30.0) as client:
            base_url = "https://search.api.hubmapconsortium.org/v3"
            url = f"{base_url}/{endpoint.lstrip('/')}"
            response = await client.get(url, params=query_param)
            response.raise_for_status()
            return response.json()
    except Exception as e:
        print(f"❌ HuBMAP API request failed: {e}")
        return None
    
def structured_response(data=None, message="success", status="success"):
    """
    Create a consistent response format for MCP tools.

    Args:
        data (Any): The actual data payload (list, dict, str, etc.)
        message (str): A short message about the result.
        status (str): Either 'success' or 'error'.

    Returns:
        dict: Standardized response object.
    """
    return {
        "status": status,
        "context": message,
        "content": data if data is not None else [],
    }

async def make_hubmap_endpoint_request(endpoint: str, query_body: dict) -> Optional[dict]:
    """
    Query HuBMAP API via /v3/search and return JSON response.

    Args:
        endpoint (str): HuBMAP API endpoint (e.g., "search", etc.).
        query_body (dict): Dictionary of query parameters (sent in POST body).

    Returns:
        Optional[dict]: Parsed JSON response from the HuBMAP API, or None if an error occurred.
    """
    try:
        async with httpx.AsyncClient(timeout=30.0) as client:
            url = f"https://search.api.hubmapconsortium.org/v3/{endpoint}"
            headers = {"Content-Type": "application/json"}
            response = await client.post(url, headers=headers, json=query_body)
            response.raise_for_status()
            return response.json()
    except Exception as e:
        print(f"❌ HuBMAP API request failed: {e}")
        return None

async def search_metadata_by_organ(organ: str, size: int = 20) -> Optional[dict]:
    """
    Search HuBMAP metadata entries for a given organ using the HuBMAP API.

    Args:
        organ (str): Name of the organ to search for (e.g., "kidney").
        size (int): Maximum number of entries to return (default: 20).

    Returns:
        Optional[dict]: Search results containing metadata for the specified organ.
    """
    query = {
        "q": f"organ:{organ}",
        "size": size,
        "sort": "last_modified_timestamp:desc"
    }

    result = await make_hubmap_endpoint_request("search", query)
    
    if result is None or "hits" not in result:
        return {"status": "error", "message": "No results or request failed."}
    
    return {
        "status": "success",
        "count": len(result["hits"]["hits"]),
        "results": [
            {
                "entity_id": hit["_id"],
                "organ": hit["_source"].get("organ"),
                "donor_id": hit["_source"].get("donor", {}).get("hubmap_id"),
                "last_modified": hit["_source"].get("last_modified_timestamp"),
                "sample_type": hit["_source"].get("sample_category"),
            }
            for hit in result["hits"]["hits"]
        ]
    }

In [16]:
result = await search_metadata_by_organ("heart")

❌ HuBMAP API request failed: Client error '400 Bad Request' for url 'https://search.api.hubmapconsortium.org/v3/search'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400


In [11]:
xx.get("results")

AttributeError: 'coroutine' object has no attribute 'get'